# Fashion Mnist
Fashion MNist es un dataset que contiene imágenes de ropa con fines académicos de aprender a crear un sistema de clasificación.

## Redes Neuronales Completamente Conectadas
Este es el sistema que usaremos para resolver el problema.

In [ ]:
# Importando las librerías
from fashion_mnist_master.utils.mnist_reader import load_mnist
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as op
import itertools

In [ ]:
## Importando la data
X_train, y_train = load_mnist('fashion_mnist_master/data/fashion', kind='train')
X_test, y_test = load_mnist('fashion_mnist_master/data/fashion', kind='t10k')

Tras ver la data por anticipado, se puede notar que existen valores entre 0 y 255 para identificar la intensidad de color de bits en la imagen. Debido a que los números están demasiado grandes. Por éso se procede a aplicarles una reducción a través de la variable _chiquitolina_. 

In [ ]:
#### Transformación de X_train
chiquitolina = (1/255) # Para achiquitar los valores
X_train = X_train * (chiquitolina)
X_train = X_train.astype(np.float16)

In [ ]:
np.random.seed(0) # Semilla de aleatoriedad constante

In [ ]:
## X_train tiene la siguiente forma
### (observaciones, atributos)
### Cada elemento del array es una observación. 
### Según mi documento oficial juandieguístico tiene la forma n*m
L = 6 # Capas desde la X hasta la Y
### One-hot enconding para Y
y_train_one_hot = np.zeros((y_train.size, y_train.max()+1))
y_train_one_hot[np.arange(y_train.size),y_train] = 1

### Funciones propias de la Red Neuronal y capas de activación
Las funciones que servirán como parámetros para la aplicación de _optimize_.

In [ ]:
## FUNCIONES IMPORTANTES
def funcion_sigmoide(x, tetas):
    """Realiza la función sigmoide
        para cualquier parámetro teta
    """
    return np.power((1 + np.exp(-1 * np.dot(x, tetas.T)) ), -1).astype(np.float16)

In [ ]:
def funcion_costo(tetas, figuras, x, y):
    """Realiza la función de costo
        para cualquier parámetro teta
    """
    tetas = deschatar_thetas(tetas, figuras)
    neuronas = forward_propagation(tetas, x) # Multiplicación de tetas
    return np.sum(np.multiply(y, np.log( neuronas[L-1] )) + np.multiply((1-y), np.log(1 - neuronas[L-1]))  ) / -len(y)

In [ ]:
def precision(y_pred, y):
    """Calcula la precisión del modelo con base
        a los valores reales y predichos de Y 
    """
    enconding = 0
    for i in range(len(y_pred)):
        valorMax = max(y_pred[i])
        valor = np.where(y_pred[i] == valorMax)
        if(valor == y[i]):
            enconding += 1 
        
    return enconding / len(y)

### Variables de uso general para la red neuronal
Aquí irán las neuronas, tamaño del modelo, constantes multiplicadoras y capa de _bias_.

In [ ]:
thetas = []
#### Cada elemento contiene un arreglo de thetas. 
####      Para el primero son c*m según el documento juandieguístico.
#### Después serán 5*5 y el último será 10*6
thetas1 = np.random.rand(5, 785).astype(np.float16) # X a capa1
thetas2 = np.random.rand(5, 6).astype(np.float16) # capa1 a 2
thetas3 = np.random.rand(5, 6).astype(np.float16) # capa2 a 3
thetas4 = np.random.rand(5, 6).astype(np.float16) # capa3 a 4
thetas5 = np.random.rand(10, 6).astype(np.float16) # capa4 a resultado
## agregando thetas
thetas.append(thetas1)
thetas.append(thetas2)
thetas.append(thetas3)
thetas.append(thetas4)
thetas.append(thetas5)

figuras = [(5, 785), (5, 6), (5, 6), (5, 6), (10, 6)]

unos = np.ones((len(X_train), 1)).astype(np.float16) ## agregar columna de unos

### Funciones de transformación de thetas para Scipy optimize
Los multiplicadores deben de entregarse en un arreglo plano, por lo que hay funciones para aplanarlo y desaplanarlo.

In [ ]:
def achatar_thetas(tetas):
    achatadas = np.array([])
    for i in tetas:
        achatadas = np.append(achatadas, i.flatten())
    achatadas.flatten()
    return achatadas

def deschatar_thetas(tetas, figura):
    deschatadas = []
    inicio = 0
    fin = 0
    for i in figura:
        fin = inicio + i[1]*i[0]
        deschatadas.append(np.array(tetas[inicio:fin]).reshape(i))
        inicio = fin
    return deschatadas

### Funciones NN

In [ ]:
def forward_propagation(tetas, x):
    unos = np.ones((len(x), 1))
    neuronas = [None] * L
    neuronas[0] = X_train ## agregando la primera capa
    for i in range(len(tetas)):
        ###### La siguiente neurona
        # print(i)
        # print(neuronas[i].shape)
        # print((np.append(neuronas[i], unos, axis=1)).shape)
        # print(neuronas[i])
        # capa_mas_bias = np.append(neuronas[i], unos, axis=1) # neurona actual más capa de unos (bias)
        neuronas[i + 1] = funcion_sigmoide(
            np.append(neuronas[i], unos, axis=1), # neurona actual más capa de unos (bias)
            tetas[i]) # Thetas actuales (ya incluyen la capa bias)
    return neuronas

In [ ]:
def back_propagation(tetas, figuras, x, y):
    tetas = deschatar_thetas(tetas, figuras)
    # Back propagation
    # Forward Propagation (Paso 2.2)
    neuronas = forward_propagation(tetas, x)
    # Paso 1 y 2.1
    Deltas = [i * 0.0 for i in tetas]
    deltas = [i * 0.0 for i in neuronas]
    deltas[L - 1] = neuronas[L - 1] - y # Paso 2.3 del documento de Samuel Chávez
    for l in reversed(range(0, L - 2)):
        # Paso 2.4
        deltas[l] = np.multiply(
                np.dot(tetas[l].T[:-1], deltas[l + 1].T).T,
                np.multiply(
                    neuronas[l],
                    (1 - neuronas[l])
                    ))
        # Paso 2.5
        Deltas[l] = Deltas[l] + np.dot(deltas[l + 1].T,
                                       np.append(neuronas[l],
                                                 unos, axis=1))
        # Paso 3
        Deltas[l] = Deltas[l] * (1/len(x))
    tetas = achatar_thetas(tetas)
    return achatar_thetas(Deltas)

## Implementación con Optimize

In [ ]:
thetas = achatar_thetas(thetas)
costo = funcion_costo(thetas, figuras, X_train, y_train_one_hot)
print("Costo actual: (antes de optimizar)", costo)
# print("Precisión antes de optimizar: ", precision(neuronas[L - 1], y_train))

In [ ]:
res = op.minimize(
    fun=funcion_costo,
    x0=thetas,
    args=(figuras, X_train, y_train_one_hot),
    method='L-BFGS-B',
    jac=back_propagation,
    options={'disp': True, 'maxiter': 1300})

In [ ]:
thetas = res.x
costo = funcion_costo(thetas, figuras, neuronas, X_train, y_train_one_hot)
neuronas = forward_propagation(thetas, X_train)
print(costo)
print(precision(neuronas[L - 1], y_train))